In [ ]:
#!/usr/bin/env python

try:
    import gi

    gi.require_version("NumCosmo", "1.0")
    gi.require_version("NumCosmoMath", "1.0")
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import numpy as np
import sys

sys.path.insert(0, "../../scripts")

import pyccl as ccl
from nc_ccl import create_nc_obj, ccl_cosmo_set_high_prec

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs
import pandas as pd

%matplotlib inline
catalog = "skysim5000_v1.1.1"
skysim_cat = GCRCatalogs.load_catalog(catalog)
cosmo_ss = skysim_cat.cosmology

In [ ]:
cosmo = Nc.HICosmoDEXcdm()
reion = Nc.HIReionCamb.new()
prim = Nc.HIPrimPowerLaw.new()

cosmo.add_submodel(reion)
cosmo.add_submodel(prim)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFunc.new_from_name("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new(tf)

# psml = Nc.PowspecMLCBE.new ()
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()


cosmo.props.H0 = cosmo_ss.H0.value
cosmo.props.Omegab = cosmo_ss.Ob0
cosmo.props.Omegac = cosmo_ss.Odm0
cosmo.props.Omegax = cosmo_ss.Ode0

cosmo.omega_x2omega_k()
cosmo.param_set_by_name("Omegak", 0.0)

prim.props.n_SA = cosmo_ss.n_s
print(cosmo_ss.sigma8, cosmo.sigma8(psf), cosmo.Omega_k0())

old_amplitude = math.exp(prim.props.ln10e10ASA)
prim.props.ln10e10ASA = math.log(
    (cosmo_ss.sigma8 / cosmo.sigma8(psf)) ** 2 * old_amplitude
)
print(cosmo_ss.sigma8, cosmo.sigma8(psf))

In [ ]:
# CosmoSim_proxy model
# M_0, z_0
theta_pivot = [3e14 / 0.71, 0.6]
# \mu_0, a_\mu^z, a_\mu^M
theta_mu = [3.19, -0.7, 2]
# \sigma_0, a_\sigma^z, a_\sigma^M
theta_sigma = [0.33, 0.0, -0.08]
# Richness object

area = 439.78987
lnRl = 0.0
lnRu = 2.0
zl = 0.0
zu = 1.0

# Numcosmo_proxy model
cluster_z = Nc.ClusterRedshift.new_from_name(
    "NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (zl, zu)
)

cluster_m = Nc.ClusterMass.new_from_name(
    "NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}"
    % (3e14 / (0.71), 0.6, lnRl, lnRu)
)
cluster_m.param_set_by_name("mup0", 3.19)
cluster_m.param_set_by_name("mup1", 2 / np.log(10))
cluster_m.param_set_by_name("mup2", -0.7 / np.log(10))
cluster_m.param_set_by_name("sigmap0", 0.33)
cluster_m.param_set_by_name("sigmap1", -0.08 / np.log(10))
cluster_m.param_set_by_name("sigmap2", 0 / np.log(10))

# Nodist

# cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (math.log(10)*np.log10(1e13),math.log(10)*np.log10(1e15)))

In [ ]:
# Numcosmo Cluster Abundance

# First we need to define the multiplicity function here we will use the tinker
mulf = Nc.MultiplicityFuncTinker.new()
mulf.set_linear_interp(True)
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
# Second we need to construct a filtered power spectrum

hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area_sd(area)

ca = Nc.ClusterAbundance.new(hmf, None)
mset = Ncm.MSet.new_array([cosmo, cluster_m, cluster_z])

In [ ]:
# With proxy

ncount = Nc.DataClusterNCount.new(ca, "NcClusterRedshiftNodist", "NcClusterMassAscaso")
ncount.catalog_load("ncount_ascaso2.fits")

# True table

# ncount = Nc.DataClusterNCount.new (ca, "NcClusterRedshiftNodist", "NcClusterMassNodist")
# ncount.catalog_load ("ncount_nodist.fits")

In [ ]:
cosmo.props.Omegac_fit = True
prim.props.ln10e10ASA_fit = True

cluster_m.props.mup0_fit = True
"""
cluster_m.props.mup1_fit = True
cluster_m.props.mup2_fit = True
cluster_m.props.sigmap0_fit = True
cluster_m.props.sigmap1_fit = True
cluster_m.props.sigmap2_fit = True
"""

mset.prepare_fparam_map()

In [ ]:
ncount.set_binned(True)

In [ ]:
dset = Ncm.Dataset.new()
dset.append_data(ncount)

In [ ]:
lh = Ncm.Likelihood(dataset=dset)
fit = Ncm.Fit.new(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_FORWARD
)
fit.log_info()

In [ ]:
#
# Setting single thread calculation.
#
Ncm.func_eval_set_max_threads(150)
Ncm.func_eval_log_pool_stats()

#
# Additional functions as we want the chains for sigma8 and Omegam, which are derived parameters
#
mfunc_oa = Ncm.ObjArray.new()

mfunc_Omegam = Ncm.MSetFuncList.new("NcHICosmo:Omega_m0", None)
mfunc_sigma8 = Ncm.MSetFuncList.new("NcHICosmo:sigma8", psf)


mfunc_oa.add(mfunc_sigma8)
mfunc_oa.add(mfunc_Omegam)

print(mfunc_sigma8.eval0(mset))
print(mfunc_Omegam.eval0(mset))

#
# New Gaussian prior to provide the initial points for the chain.
# It was created with size 0 (number of parameters), but once
# initialized with mset the correct size is assigned.
#
# The initial sampler will use a diagonal covariance with the
# diagonal terms being the parameters scale set by each model.
#
init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)  # 1

#
# Creates the ESMCMC walker object, this object is responsible
# for moving the walkers in each interation, the stretch move
# is affine invariant and therefore gives good results even for
# very correlated parametric space.
#
sampler = "apes"
# sampler  = 'stretch'
nwalkers = int(math.ceil(500))  # 500
ssize = 15000  # 1000000

if sampler == "apes":
    walker = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())
elif sampler == "stretch":
    walker = Ncm.FitESMCMCWalkerStretch.new(nwalkers, mset.fparams_len())
#
# The methods below set the walk scale, which controls the size of the
# step done between two walkers and circumscribe the walkers inside
# the box defined by the parameters inside the mset object.
#
# walker.set_scale (3.0)
# walker.set_box_mset (mset)
#
# Initialize the ESMCMC object using the objects above. It will
# use 50 walkers, i.e., each point in the MCMC chain contains
# 50 points in the parametric space. Each step uses the last point
# in the chain (the last 50 parametric points) to calculate the
# proposal points.
#
esmcmc = Ncm.FitESMCMC.new_funcs_array(
    fit, nwalkers, init_sampler, walker, Ncm.FitRunMsgs.SIMPLE, mfunc_oa
)

#
# These methods enable the auto-trim options on ESMCMC. This option
# makes the sampler check the chains' health and trim any unnecessary
# burn-in part. We set the number of divisions to 100 so we test the
# chains in blocks of n/100. The last method asserts that each 2min
# the catalog will be checked.
#
# esmcmc.set_auto_trim (True)
# esmcmc.set_auto_trim_div (100)
# esmcmc.set_max_runs_time (2.0 * 60.0)
# esmcmc.set_nthreads (4)

# With self calibration
esmcmc.set_data_file("mcmc_ascaso_binned.fits")
# without self calibration
# esmcmc.set_data_file ("mcmc_ascaso_binned_noself.fits")
# esmcmc.set_data_file ("mcmc_nodist_binned.fits")

esmcmc.set_nthreads(150)

#
# Running the esmcmc, it will first calculate 1000 points, after that
# it will estimate the error in the parameters mean. Using the current
# errors the algorithm tries to calculated how many extra steps are
# necessary to obtain the required error `10^-3' in every parameters,
# and it will run such extra steps. It will repeat this procedure
# until it attains the required error in every parameter.
#
#

esmcmc.start_run()
# esmcmc.run (ssize / nwalkers)
# esmcmc.run (10)
esmcmc.run_lre(50, 1.0e-3)
esmcmc.end_run()

#
# Calculates the parameter means and covariance and set it into
# the fit object and then print.
#
esmcmc.mean_covar()
fit.log_covar()

In [ ]:
ntests = 100.0
nwalkers = 500
burnin = 30
mcat = Ncm.MSetCatalog.new_from_file_ro("mcmc_nodist_binned.fits", nwalkers * burnin)

mcat.log_current_chain_stats()
mcat.calc_max_ess_time(ntests, Ncm.FitRunMsgs.SIMPLE)
mcat.calc_heidel_diag(ntests, 0.0, Ncm.FitRunMsgs.SIMPLE)

mset.pretty_log()
mcat.log_full_covar()
mcat.log_current_stats()

be, post_lnnorm_sd = mcat.get_post_lnnorm()
lnevol, glnvol = mcat.get_post_lnvol(0.6827)

Ncm.cfg_msg_sepa()
print(
    "# Bayesian evidence:                                 % 22.15g +/- % 22.15g"
    % (be, post_lnnorm_sd)
)
print("# 1 sigma posterior volume:                          % 22.15g" % lnevol)
print("# 1 sigma posterior volume (Gaussian approximation): % 22.15g" % glnvol)

In [ ]:
# proxy noself 14 min 30 segundos 1-sigma posterior volume -10.41
# proxy self 42 min 58 segundos 1-sigma posterior volume -14.39
# nodist 13 min 56 segundos 1-sigma posterior volume -10.83